In [1]:
import yeelight as yl
from yeelight import discover_bulbs
import time
from pynput import mouse

In [9]:
discover_bulbs()

[{'ip': '192.168.1.230',
  'port': 55443,
  'capabilities': {'id': '0x00000000158acbe2',
   'model': 'color4',
   'fw_ver': '34',
   'support': 'get_prop set_default set_power toggle set_bright set_scene cron_add cron_get cron_del start_cf stop_cf set_ct_abx adjust_ct set_name set_adjust adjust_bright adjust_color set_rgb set_hsv set_music udp_sess_new udp_sess_keep_alive udp_chroma_sess_new',
   'power': 'on',
   'bright': '100',
   'color_mode': '2',
   'ct': '5550',
   'rgb': '2544895',
   'hue': '192',
   'sat': '85',
   'name': 'bedroom1'}}]

In [2]:
def get_bulbs(names):
    bulb_dicts = discover_bulbs()
    bulbs = []
    for bulb_dict in bulb_dicts:
        if bulb_dict['capabilities']['name'] in names:
            bulbs.append(yl.Bulb(bulb_dict['ip']))
            
    return bulbs

In [3]:
def toggle_music_mode(bulbs, state=None):
    for bulb in bulbs:
        try:
            bulb.get_properties()
        except yl.BulbException as e:
            print(e)
            return -1
        
        if state is None:
            if bulb.get_properties()['music_on'] == '0':
                target = 1
            else:
                target = '0'
        else:
            target = state
        
        if target == 1:
            method = bulb.start_music
        else:
            method = bulb.stop_music

        while bulb.get_properties()['music_on'] != target:
            o = bulb.get_properties()['music_on']
            try:
                method()
                break
            except yl.BulbException as e:
                pass
            except TypeError as e:
                print("Caught error:", e)
    return target

In [4]:
def set_color_temp(bulbs, degrees):    
    degrees =  min(6500, max(degrees, 1700))
    for bulb in bulbs:
        try:
            bulb.set_power_mode(yl.PowerMode.NORMAL)
            bulb.set_color_temp(degrees)
        except yl.BulbException as e:
            pass

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [16]:
bulbs = get_bulbs(['bedroom1'])
bulbs[0].get_properties()['music_on']

'0'

In [17]:
prev_t = time.time()
def set_color_temp_slider(temp):
    global prev_t
    
    if time.time() < prev_t + .1:
        return
    set_color_temp(bulbs, temp)
    prev_t = time.time()

toggle_music_mode(bulbs, state=1)
interact(set_color_temp_slider, temp=widgets.IntSlider(min=1500, max=6700, step=50, value=4000))

interactive(children=(IntSlider(value=4000, description='temp', max=6700, min=1500, step=50), Output()), _dom_…

<function __main__.set_color_temp_slider(temp)>

In [18]:
toggle_music_mode(bulbs, state='0')

'0'

In [147]:
bulbs

[Bulb<192.168.1.230:55443, type=BulbType.Color>]

In [37]:
target = int(bulbs[0].get_properties()['ct'])
def on_scroll(x, y, dx, dy):
    global target
#     print('Scrolled {0} at {1}'.format(
#         'down' if dy < 0 else 'up',
#         (x, y)))
    
    if dy < 0:
        target -= 50
        
    elif dy > 0:
        target += 50
        
    target = min(6700, max(target, 1700))  
    set_color_temp_slider(target)

toggle_music_mode(bulbs, 1)
# Collect events until released
if globals().get('listener'):
    listener.stop()
listener = mouse.Listener(
        on_scroll=on_scroll)
listener.start()

In [40]:
toggle_music_mode(bulbs, '0')
listener.stop()

In [41]:
bulbs

[Bulb<192.168.1.230:55443, type=BulbType.Color>]

In [39]:
bulbs[0].get_properties()

{'power': 'on',
 'bright': '100',
 'ct': '3750',
 'rgb': '2544895',
 'hue': '192',
 'sat': '85',
 'color_mode': '2',
 'flowing': '0',
 'delayoff': '0',
 'music_on': '0',
 'name': 'bedroom1',
 'bg_power': None,
 'bg_flowing': None,
 'bg_ct': None,
 'bg_bright': None,
 'bg_hue': None,
 'bg_sat': None,
 'bg_rgb': None,
 'nl_br': None,
 'active_mode': None,
 'current_brightness': '100'}

In [6]:
from utils import get_bulbs
bulbs = get_bulbs(['bedroom1'])

In [16]:
prev_t = time.time()
def set_brightness(bulbs, brightness):
    global prev_t 
    brightness = min(100, max(brightness, 0))

    # Rate limit commands
    if time.time() < prev_t + .1:
        return 0

    for bulb in bulbs:
        try:
            if brightness == 0:
                if bulb.on:
                    bulb.turn_off()
            else:
                if not bulb.on:
                    bulb.turn_on()
                bulb.set_brightness(brightness)
        except yl.BulbException as e:
            pass 
    prev_t = time.time()
    return brightness

In [19]:
target = int(bulbs[0].get_properties()['bright'])
def on_scroll(x, y, dx, dy):
    global target
#     print('Scrolled {0} at {1}'.format(
#         'down' if dy < 0 else 'up',
#         (x, y)))
    
    if dy < 0:
        target -= 5
        
    elif dy > 0:
        target += 5
        
    target = min(100, max(target, 0))  
    set_brightness(bulbs, target)

toggle_music_mode(bulbs, 1)
# Collect events until released
if globals().get('listener'):
    listener.stop()
listener = mouse.Listener(
        on_scroll=on_scroll)
listener.start()

In [20]:
toggle_music_mode(bulbs, '0')
listener.stop()

In [21]:
bulbs[0].get_properties()

{'power': 'on',
 'bright': '100',
 'ct': '3750',
 'rgb': '2544895',
 'hue': '192',
 'sat': '85',
 'color_mode': '2',
 'flowing': '0',
 'delayoff': '0',
 'music_on': '0',
 'name': 'bedroom1',
 'bg_power': None,
 'bg_flowing': None,
 'bg_ct': None,
 'bg_bright': None,
 'bg_hue': None,
 'bg_sat': None,
 'bg_rgb': None,
 'nl_br': None,
 'active_mode': None,
 'current_brightness': '100'}

In [22]:
bulbs[0].get_capabilities()

{'id': '0x00000000158acbe2',
 'model': 'color4',
 'fw_ver': '34',
 'support': 'get_prop set_default set_power toggle set_bright set_scene cron_add cron_get cron_del start_cf stop_cf set_ct_abx adjust_ct set_name set_adjust adjust_bright adjust_color set_rgb set_hsv set_music udp_sess_new udp_sess_keep_alive udp_chroma_sess_new',
 'power': 'on',
 'bright': '100',
 'color_mode': '2',
 'ct': '3750',
 'rgb': '2544895',
 'hue': '192',
 'sat': '85',
 'name': 'bedroom1'}

In [28]:
bulbs[0].get_model_specs()

{'color_temp': {'min': 1700, 'max': 6500},
 'night_light': False,
 'background_light': False}

In [31]:
bulbs[0].bulb.__dict__

{'_ip': '192.168.1.230',
 '_port': 55443,
 'effect': 'smooth',
 'duration': 300,
 'auto_on': False,
 'power_mode': <PowerMode.LAST: 0>,
 '_model': None,
 '_Bulb__cmd_id': 112,
 '_last_properties': {'power': 'on',
  'bright': '100',
  'ct': '3750',
  'rgb': '2544895',
  'hue': '192',
  'sat': '85',
  'color_mode': '2',
  'flowing': '0',
  'delayoff': '0',
  'music_on': '0',
  'name': 'bedroom1',
  'bg_power': None,
  'bg_flowing': None,
  'bg_ct': None,
  'bg_bright': None,
  'bg_hue': None,
  'bg_sat': None,
  'bg_rgb': None,
  'nl_br': None,
  'active_mode': None,
  'current_brightness': '100'},
 '_capabilities': {'id': '0x00000000158acbe2',
  'model': 'color4',
  'fw_ver': '34',
  'support': 'get_prop set_default set_power toggle set_bright set_scene cron_add cron_get cron_del start_cf stop_cf set_ct_abx adjust_ct set_name set_adjust adjust_bright adjust_color set_rgb set_hsv set_music udp_sess_new udp_sess_keep_alive udp_chroma_sess_new',
  'power': 'on',
  'bright': '100',
  'color